In [1]:
# Imports and stuff
import os
import time
import csv

#Libraries
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.feature_extraction import stop_words
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.svm import NuSVC

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold



#Powerpuff-stuff
import helpers as HL
import tolken as TO



### Running for local testing


In [15]:
# RUNNING ON TRAINING SET WITH CROSS VALIDATION

training_set_pos = "train_pos.txt" 
training_set_neg = "train_neg.txt"
test_set = "test_data.txt"

inputfiles=[training_set_pos, training_set_neg]

corpus, file_lengths=HL.create_corpus(inputfiles)

In [9]:
hedda_corpus = TO.replace_words(original_corpus)

In [22]:
#corpus = HL.creating_n_grams_cropus(2, hedda_corpus)

In [10]:
corpus = hedda_corpus

In [16]:
nr_pos_lines = 100000
nr_lines_total =200000

#Making labels, setting pos = 1 and neg = 0 
labels = np.zeros(nr_lines_total);
labels[0:nr_pos_lines]=1;
labels[nr_pos_lines:nr_lines_total]=0;

#Variables for measuring accuracy -----------------------

totalsvm = 0   # Accuracy measure 
totalMatSvm = np.zeros((2,2));  # Confusion matrix 

#Running algorithm with K-fold ---------------------------

#Creating K-fold = 10 for cross validation
kf = StratifiedKFold(n_splits=10)

for train_index, test_index in kf.split(corpus,labels):
    
    X_train = [corpus[i] for i in train_index]
    X_test = [corpus[i] for i in test_index]
    
    y_train, y_test = labels[train_index], labels[test_index]
    
    vectorizer = TfidfVectorizer(
        min_df = 10, # removing word that occure less then 10 times 
        max_df = 0.5, # remove words that are too frequent ( more then 0.8 * number of files )
        sublinear_tf=True, # scale the term frequency in logarithmic scale
        max_features = 3000,
        use_idf = True
    )
    
    train_corpus_tf_idf = vectorizer.fit_transform(X_train) 
    test_corpus_tf_idf = vectorizer.transform(X_test)

    model1 = LinearSVC()
    model1.fit(train_corpus_tf_idf,y_train)
    result1 = model1.predict(test_corpus_tf_idf)
    
    print(result1)
    
    totalMatSvm = totalMatSvm + confusion_matrix(y_test, result1)
    totalsvm = totalsvm+sum(y_test==result1)
    
print(totalMatSvm, "\n", "Total accuracy:", totalsvm/nr_lines_total)  

[ 1.  0.  0. ...,  1.  0.  0.]
[ 1.  1.  1. ...,  0.  1.  0.]
[ 1.  1.  1. ...,  0.  0.  0.]
[ 1.  1.  1. ...,  0.  1.  1.]
[ 1.  1.  1. ...,  0.  0.  0.]
[ 1.  1.  1. ...,  1.  1.  0.]
[ 1.  1.  1. ...,  0.  0.  1.]
[ 1.  1.  1. ...,  0.  0.  1.]
[ 1.  1.  1. ...,  1.  0.  0.]
[ 1.  1.  1. ...,  0.  0.  0.]
[[ 76114.  23886.]
 [ 18041.  81959.]] 
 Total accuracy: 0.790365


### Running for keggle delivery

In [18]:
start = time.time()

training_set_pos = "train_pos_full.txt" 
training_set_neg = "train_neg_full.txt"
test_set = "test_data.txt"

inputfiles=[training_set_pos, training_set_neg]

corpus, file_lengths=HL.create_corpus(inputfiles)

#Couning nr. of lines in pos and neg training_set
nr_pos_lines = 1250000
nr_neg_lines = 1250000

#### CREATING TEST CORPUS ####
test_corpus, test_coprus_length = HL.create_corpus(['test_data.txt'])

labels = HL.create_labels(nr_neg_lines, nr_pos_lines)

vectorizer = TfidfVectorizer(
    min_df=9, # removing word that occure less then X times 
    max_df = 0.8, # remove words that are too frequent, in X*100 % of the files
    sublinear_tf=True, # scale the term frequency in logarithmic scale
    use_idf =True
)

#CLASSIFYING WITH SVM
model = LinearSVC()

elapsed = time.time()

print ("Time in min:", (elapsed - start) / 60 )


Time in min: 0.018616263071695963


In [19]:
start = time.time()

#VECTORIZING ( Basically the preprocessing, see: TfidfVectorizer)
train_corpus_fittrans = vectorizer.fit_transform(corpus) 
test_corpus_trans = vectorizer.transform(test_corpus)

elapsed = time.time()

print ("Time in min:", (elapsed - start) / 60 )

Time in min: 0.7540553132692973


In [20]:
start = time.time()

# FITTING THE MODEL
model.fit(train_corpus_fittrans, labels)

elapsed = time.time()
print ("Time in min:", (elapsed - start) / 60 )

Time in min: 14.595923177401225


In [21]:
start = time.time()

#PREDICTING
submission_prediction = model.predict(test_corpus_trans)
print(submission_prediction)

elapsed = time.time()
print ("Time in min:", (elapsed - start) / 60 )

[ 0.  0.  0. ...,  0.  1.  0.]
Time in min: 5.0127506256103516e-05


In [27]:
submission_prediction = [-1 if val==0 else val for val in submission_prediction]

In [26]:
start = time.time()

#CREATING SUBMISSION
ids = list(range(1,nr_lines_total+1))
y_pred = submission_prediction
name = "keggle_submission_idftf.csv"

HL.create_csv_submission(ids, y_pred, name)

elapsed = time.time()
print ("Time in min:", (elapsed - start) / 60 )

Time in min: 0.0011864821116129556


# EVERYTHING BELOW THIS IS JUST A MESS! DON'T GO THERE IF YOU DON'T HAVE TOO!

## Running on test-data with GRID-SEARCH on parameters for tokenizing 

In [ ]:
# RUNNING ON TRAINING SET WITH CROSS VALIDATION

training_set_pos = "train_pos.txt" 
training_set_neg = "train_neg.txt"
test_set = "test_data.txt"

#Couning nr. of lines in pos and neg training_set
nr_pos_lines = 0
nr_neg_lines = 0

#Creating corpus ---------------------------------------
corpus = []

#Adding lines from positive training set to CORPUS
with open(training_set_pos,'rb') as infile:
    for line in infile:
        corpus.append(line)

nr_pos_lines = len(corpus)
        
#Adding lines from negative training set to CORPUS
with open(training_set_neg,'rb') as infile:
    for line in infile:
        corpus.append(line)
        
nr_neg_lines = len(corpus) - nr_pos_lines

#Finding tot nr. of lines
nr_lines_total = nr_pos_lines + nr_neg_lines

#Creating labels-array  ---------------------------------    

#Making labels, setting pos = 1 and neg = 0 
labels = np.zeros(nr_lines_total);
labels[0:nr_pos_lines]=1;
labels[nr_pos_lines:nr_lines_total]=0;

#Variables for measuring accuracy -----------------------

totalsvm = 0   # Accuracy measure 
totalMatSvm = np.zeros((2,2));  # Confusion matrix 

#Running algorithm with K-fold ---------------------------

 
min_df_list = list(range(10,13))
max_df_list = np.linspace(0.3, 2.0, num=15)

scores = []
best_score = 0
best_min_df = 0 
best_max_df = 0

for min_df in min_df_list:
    for max_df in max_df_list:
        
        #Creating K-fold for cross validation
        kf = StratifiedKFold(n_splits=2)
        
        totalsvm = 0

        for train_index, test_index in kf.split(corpus,labels):

            X_train = [corpus[i] for i in train_index]
            X_test = [corpus[i] for i in test_index]

            y_train, y_test = labels[train_index], labels[test_index]

            vectorizer = TfidfVectorizer(
                min_df = min_df, # removing word that occure less then X times 
                max_df = max_df, # remove words that are too frequent ( more then 0.8 * number of files )
                sublinear_tf=True, # scale the term frequency in logarithmic scale
                use_idf =True
                #stop_words = 'english', # Removing stop-words
            )

            train_corpus_tf_idf = vectorizer.fit_transform(X_train) 
            test_corpus_tf_idf = vectorizer.transform(X_test)

            model1 = LinearSVC()
            model1.fit(train_corpus_tf_idf,y_train)
            result1 = model1.predict(test_corpus_tf_idf)
            
                
            #totalMatSvm = totalMatSvm + confusion_matrix(y_test, result1)
            totalsvm = totalsvm+sum(y_test==result1)

        score = totalsvm / nr_lines_total
        scores.append(score)
        
        if(score > best_score):
            best_min_df = min_df
            best_max_df = max_df
            best_score = score
        
            print("Best score: ", best_score, "\n",
                  "With min_df:", min_df, " and ",
                  "Max_df:", max_df)
            

## STUPID N-GRAM IMPLEMENTATION, K-fold validated

In [ ]:
start = time.time()
# RUNNING ON TRAINING SET WITH CROSS VALIDATION

training_set_pos = "train_pos_full.txt" 
training_set_neg = "train_neg_full.txt"
test_set = "test_data.txt"

#Couning nr. of lines in pos and neg training_set
nr_pos_lines = 0
nr_neg_lines = 0


def add_n_grams(tweet, n):

    n_grams = []
    words = tweet.split()
    for i in range(len(words)-1):        
        n_grams.append(b''.join([words[i], b'-', words[i+1]]))
    words.extend(n_grams)
    return b'-'.join(words)

#Creating corpus ---------------------------------------
corpus = []

n_gram = 4

#Adding lines from positive training set to CORPUS
with open(training_set_pos,'rb') as infile:
    for line in infile:
        line = add_n_grams(line, n_gram)
        corpus.append(line)

nr_pos_lines = len(corpus)
        
#Adding lines from negative training set to CORPUS
with open(training_set_neg,'rb') as infile:
    for line in infile:
        line = add_n_grams(line, n_gram)
        corpus.append(line)
        
nr_neg_lines = len(corpus) - nr_pos_lines

#Finding tot nr. of lines
nr_lines_total = nr_pos_lines + nr_neg_lines

#Creating labels-array  ---------------------------------    

#Making labels, setting pos = 1 and neg = 0 
labels = np.zeros(nr_lines_total);
labels[0:nr_pos_lines]=1;
labels[nr_pos_lines:nr_lines_total]=0;

#Variables for measuring accuracy -----------------------

totalsvm = 0   # Accuracy measure 
totalMatSvm = np.zeros((2,2));  # Confusion matrix 

#Running algorithm with K-fold ---------------------------
end = time.time()

print("time in min:", (end-start)/60)
 

ONLY run the cell below if you want to CROSS VALIDATE TO FIND BEST PARAMS, if not skip next cell

In [ ]:
min_df_list = list(range(9,14))
max_df_list = np.linspace(0.8, 1.1, num=2)

scores = []
best_score = 0
best_min_df = 0 
best_max_df = 0

run = 0

start = time.time()

for min_df in min_df_list:
    for max_df in max_df_list:
        
        run += 1
        
        start = time.time()
        
    
        #Creating K-fold for cross validation
        kf = StratifiedKFold(n_splits=2)
        
        totalsvm = 0

        for train_index, test_index in kf.split(corpus,labels):

            X_train = [corpus[i] for i in train_index]
            X_test = [corpus[i] for i in test_index]

            y_train, y_test = labels[train_index], labels[test_index]

            vectorizer = TfidfVectorizer(
                min_df = min_df, # removing word that occure less then X times 
                max_df = max_df, # remove words that are too frequent ( more then 0.8 * number of files )
                sublinear_tf=True, # scale the term frequency in logarithmic scale
                use_idf =True
                #stop_words = 'english', # Removing stop-words
            )

            train_corpus_tf_idf = vectorizer.fit_transform(X_train) 
            test_corpus_tf_idf = vectorizer.transform(X_test)

            model1 = LinearSVC()
            model1.fit(train_corpus_tf_idf,y_train)
            result1 = model1.predict(test_corpus_tf_idf)
            
                
            #totalMatSvm = totalMatSvm + confusion_matrix(y_test, result1)
            totalsvm = totalsvm+sum(y_test==result1)

        score = totalsvm / nr_lines_total
        scores.append(score)
        
        if(score > best_score):
            best_min_df = min_df
            best_max_df = max_df
            best_score = score
        
            print("\n","Best score: ", best_score, "\n",
                  "With min_df:", min_df, " and ",
                  "Max_df:", max_df,"\n")
        
        end = time.time()
        print("Run: ", run, "Time in sec: ", end-start)
        

Pick best parameters from gird-search above

In [ ]:
start = time.time()

#### CREATING TEST CORPUS ####

test_corpus = []

with open(test_set,'rb') as infile:
    for line in infile:
        test_corpus.append(line)
        
        
##############################

#Making pos/neg labels, setting pos = 1 and neg = 0 
labels = np.zeros(nr_lines_total);
labels[0:nr_pos_lines]=1;
labels[nr_pos_lines:nr_lines_total]=-1;

vectorizer = TfidfVectorizer(
    min_df=9, # removing word that occure less then 10 times 
    max_df = 0.8, # remove words that are too frequent ( more then 0.8 * number of files )
    sublinear_tf=True, # scale the term frequency in logarithmic scale
    use_idf =True
    #stop_words = 'english' # Removing stop-words
)

end = time.time()

print("Time in min:", end-start)


In [ ]:
#CLASSIFYING WITH SVM
model = LinearSVC()

In [ ]:
start = time.time()

#VECTORIZING ( Basically the preprocessing, see: TfidfVectorizer)
train_corpus_fittrans = vectorizer.fit_transform(corpus) 
test_corpus_trans = vectorizer.transform(test_corpus)

elapsed = time.time()

print ("Time in min:", (elapsed - start) / 60 )

In [ ]:
start = time.time()

# FITTING THE MODEL
model.fit(train_corpus_fittrans, labels)

elapsed = time.time()
print ("Time in min:", (elapsed - start) / 60 )

In [ ]:
start = time.time()

#PREDICTING
submission_prediction = model.predict(test_corpus_trans)

elapsed = time.time()
print ("Time in min:", (elapsed - start) / 60 )

In [ ]:
start = time.time()

#CREATING SUBMISSION
ids = list(range(1,nr_lines_total+1))
y_pred = submission_prediction
name = "keggle_submission_4.csv"

create_csv_submission(ids, y_pred, name)

elapsed = time.time()
print ("Time in min:", (elapsed - start) / 60 )

In [ ]:
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in csv format for submission to kaggle
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})